In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv


In [4]:
trainning_df = pd.read_csv('../input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv')

In [5]:
pd.set_option('max_colwidth', 800)
trainning_df.head(5)

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monda...","Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed ."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,"(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as ""The Milk Man,"" of using his role as a police officer to help the drug trafficking organization in exchange for money and gifts, including a Rolex watch. In one instance, the complaint alleges, Mata arranged to pay two assassins to kill rival drug dealers. The killers would pose as cops, pulling over their targets before shooting them, according to the compla...","Criminal complaint: Cop used his role to help cocaine traffickers .\nRalph Mata, an internal affairs lieutenant, allegedly helped group get guns .\nHe also arranged to pay two assassins in a murder plot, a complaint alleges ."
2,00027e965c8264c35cc1bc55556db388da82b07f,"A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years. Craig Eccleston-Todd, 27, was driving home from a night at a pub when he received a text message. As he was reading or replying to it, he veered across the road while driving round a bend and smashed into Rachel Titley’s car coming the other way. Craig Eccleston-Todd, 27 (left) was using his mobile phone when he crashed head-on into the car being driven by Rachel Titley, 28 (right). She died later from her injuries . The head-on crash took place in October 2013. Mr Eccleston-Todd's car was barely recognisable (pictured) Police said Eccleston-Todd had drunk at least three or four pints of beer before getting behind the wheel. He was found guilty of causing death by d...","Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .\nWas using phone when he veered across road in Yarmouth, Isle of Wight .\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital .\nPolice say he would have been over legal drink-drive limit at time of crash .\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving ."
3,0002c17436637c4fe1837c935c04de47adb18e9a,"(CNN) -- With a breezy sweep of his pen President Vladimir Putin wrote a new chapter into Crimea's turbulent history, committing the region to a future returned to Russian domain. Sixty years prior, Ukraine's breakaway peninsula was signed away just as swiftly by Soviet leader Nikita Khrushchev. But dealing with such a blatant land grab on its eastern flank won't be anywhere near as quick and easy for Europe's 28-member union. Because, unlike Crimea's rushed referendum, everyone has a say. After initially slapping visa restrictions and asse

In [6]:
MAX_LEN = 512
SUMMARY_LEN = 150
TRAINNING_SIZE = 5000

In [7]:
trainning_df = trainning_df.iloc[0:TRAINNING_SIZE,:].copy()

trainning_article_ls = list(trainning_df['article'])
trainning_highlight_ls = list(trainning_df['highlights'])

del trainning_df

In [8]:
from random import sample

for index in sample(list(np.arange(len(trainning_article_ls))),3):
    print('Original Text : ')
    print(trainning_article_ls[index])

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])
    
    print('===========================================================================================================\n\n')

Original Text : 
(CNN) -- A single phone call by Osama bin Laden's trusted courier tipped off U.S. officials to his Pakistan compound, ultimately leading to the raid that killed the al Qaeda leader, a senior Pakistani intelligence official told CNN Saturday. The telephone call the courier made was "not the final one -- it was the initial piece of evidence" that sparked the focus on the compound in Abbottabad, the official said. Four years ago, U.S. officials uncovered the identity of a trusted bin Laden courier -- later identified as a Kuwaiti named Abu Ahmad -- whom they believed was living with and protecting the al Qaeda leader. The Washington Post, citing U.S. intelligence officials, reported Friday that Americans had intercepted a "catch-up phone call" Ahmad took from an old friend. "This is where you start the movie about the hunt for bin Laden," one U.S. official briefed on the intelligence-gathering leading up to the early Monday raid on the compound told the Washington Post. T

In [9]:
print(len(trainning_article_ls))
print(len(trainning_highlight_ls))

5000
5000


In [10]:
df = pd.DataFrame(columns=['target_text','source_text'])
df['target_text'] = trainning_highlight_ls
df['source_text'] = ['summarize: '+item for item in trainning_article_ls]

In [11]:
!pip install simplet5 -q

In [12]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

In [13]:
MAX_EPOCHS = 5

In [14]:
model.train(train_df=df[0:(int)(0.7*TRAINNING_SIZE)],
            eval_df=df[(int)(0.7*TRAINNING_SIZE):TRAINNING_SIZE], 
            source_max_token_len=MAX_LEN, 
            target_max_token_len=SUMMARY_LEN, 
            batch_size=8, max_epochs=MAX_EPOCHS,use_gpu=True)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [15]:
import os
model_path = ''
rootdir = 'outputs/'
for it in os.scandir(rootdir):
    if it.is_dir():
        if 'simplet5-epoch-'+(str)(MAX_EPOCHS-1) in it.path:
            model_path = it.path
            print(model_path)

outputs/simplet5-epoch-4-train-loss-1.0961-val-loss-1.5404


In [16]:
model.load_model("t5","./"+model_path, use_gpu=True)

In [17]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

no_tune_model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device('cpu')

In [ ]:
from random import sample

for index in sample(list(np.arange(len(trainning_article_ls))),5):
    print('Original Text : ')
    print(trainning_article_ls[index])

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])

    print('\n\nFine tuned Predicted Summary Text : ')
    print(model.predict(trainning_article_ls[index]))
    
    print('\n\nNot Fine tuned Predicted Summary Text : ')
    preprocess_text = trainning_article_ls[index].strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    summary_ids = no_tune_model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=SUMMARY_LEN,
                                        early_stopping=True)
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(output)
    print('===========================================================================================================\n\n')

Token indices sequence length is longer than the specified maximum sequence length for this model (857 > 512). Running this sequence through the model will result in indexing errors


Original Text : 
By . Hayley O'keeffe . PUBLISHED: . 05:23 EST, 31 October 2013 . | . UPDATED: . 12:53 EST, 31 October 2013 . When artist Alan Dodd was asked by friends to paint a mural on a blank 50-foot-long wall at their home he transformed it into an optical illusion. Friendly faces behind windows and lifelike archways leading to landscapes are painted onto the cottage, in Eye, Suffolk. And the images give an illusion of a make-believe world within. Nationally-renowned artist Alan Dodd stands with the unusual mural that he painted on the side of a friend's home in Eye, Suffolk . The largest painting in the Eye mural is a blacksmith's forge, which takes in the skyline features of the town's church and castle. Quirky characters and also animals feature on other parts of the house side. More... The picture you never thought you'd see... Anna Wintour eating a burger and fries! Artist gives Vogue editor and her fashion friends a calorie-laden makeover . Rare Italian 'San Gennaro' collec

Token indices sequence length is longer than the specified maximum sequence length for this model (859 > 512). Running this sequence through the model will result in indexing errors


["Alan Dodd painted the unusual mural on a friend's home in Eye, Suffolk. The 71-year-old spent 55 days up a ladder to complete the work. Mr Dodd says it is an homage to the market town's history."]


Not Fine tuned Predicted Summary Text : 
artist spent 55 days up a ladder painting the 50-foot-long wall. he says it is an homage to the history of the market town in eye, suffolk - and its history is reflected in the work 'against direct sunlight'


Original Text : 
Jailed: Joshua Sadler, 21, was sentenced to 12 months after admitting dangerous driving . The mother of a teenager who was killed when a friend crashed his car just four days after passing his test has spoken of her shock after the driver was jailed for only 12 months. Joshua Sadler, 21, lost control of his silver Renault Clio on a 60mph country road and smashed into a tree, killing front seat passenger Mikey Maguire, 19. Sadler had previously been convicted of a . string of serious motoring offences and twice banned from the

In [ ]:
# !pip install rouge-score

In [ ]:
# from rouge_score import rouge_scorer
# scores=[]
# scorer = rouge_scorer.RougeScorer(['rouge1'])
# scorer.score(trainning_article_ls[index],trainning_highlight_ls[index])
# scores.append(scorer.score(trainning_article_ls[index],trainning_highlight_ls[index]))

In [ ]:
scores

In [ ]:
type(model)

In [ ]:
import joblib
joblib.dump(model , "final_model.pkl")

In [ ]:
new = joblib.load("./final_model.pkl")
type(new)

In [ ]:
new.predict()